In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import AutoTokenizer

class BankiDataset(Dataset):
    def __init__(self, tokenizer: AutoTokenizer, dataset_path: str = "../data/processed/promsvyazbank_reviews.csv"):
        df = pd.read_csv(dataset_path)
        tokenized_
        for idx, row in tqdm(df, total=len(df)):
            
            tokenizer()
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        input_text = "Название: {}\nОценка:{}\n\n{}".format(row["title"], row["grattitue"], row["text"])


In [ ]:
import datasets

datasets.load("../data/processed/promsvyazbank_reviews.csv")